# Create SHETRAN Elevation Data
*Ben Smith | 12/12/2025*

This script is designed to take online downloads and reconfigure them into raster layers that can be used to setup SHETRAN models.

Todo:
- Consider the fixes for the catchments that are below sea level and those that seem to have issues in SHETRAN simulaitons.

### Preamble

In [ ]:
import os
import zipfile
import shutil

import rasterio
import numpy as np

import pandas as pd

from rasterio.merge import merge

from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterio.transform import from_bounds
from rasterio.features import rasterize

import geopandas as gpd

import Functions_Model_Set_Up as she

root = 'I:/SHETRAN_GB_2021/02_Input_Data/01 - National Data Inputs for SHETRAN UK/'
resolution_output = 200

# Create a function for simplifying map and table data by removing duplicates:
def remove_map_df_duplicates(map_path, table_path, ID_col, duplicate_columns, output_suffix='_unique', data_format: str = '%1.1f'):
    """
    Function to remove duplicate entries in a raster and a linked dataframe.
    The duplicates are identified based on the columns in duplicate_columns, and the minimum Raster_ID is used for each group.
    ID_col: string of ID column. e.g. 'Raster_ID'
    duplicate_columns = list of column srings. e.g. ['Flow Mechanism', 'Summary']
    """

    # Read in the table and map:
    print('Reading in data...')
    table = pd.read_csv(table_path)
    map, mc, mr, mx, my, mcs, mnd, _, _ = she.read_ascii_raster(map_path, data_type=int, replace_NA=False)

    # Group using the desired columns and return the raster IDs in each group:
    print('Grouping duplicated data...')
    groups = table.groupby(duplicate_columns)[ID_col].apply(list).reset_index().Raster_ID

    # -- Now run through each group, finding the minimum Raster_ID and changing all other IDs to that in the map.

    # Run throug the groups:
    print('Running through duplicates...')
    for group in groups:
        # Find minimum ID: 
        new_ID = min(group)

        # Change the duplicated IDs to the new ID:
        for old_ID in group:
            ## Table
            # table.loc[table[ID_col] == old_ID, ID_col] = new_ID
            # Map
            map[map == old_ID] = new_ID

    # Drop duplicates from the tbale - the method above will change the IDs but will not remove duplicate rows:
    print('Dropping duplicates...')
    table.drop_duplicates(subset=duplicate_columns, keep='first', inplace=True)#.reset_index(drop=True)

    # Reset the indexes so that they run consecutively:
    print('resetting indexes...')
    counter = 1
    for old_ID in sorted(table[ID_col]):
        # Table:
        table.loc[table[ID_col] == old_ID, ID_col] = counter
        # Map
        map[map == old_ID] = counter
        counter+=1

    # Write out the map:
    print('Writing data...')
    she.write_ascii(array=map, ascii_ouput_path=map_path.replace('.asc', f'{output_suffix}.asc'),
        xllcorner=mx, yllcorner=my, cellsize=mcs, ncols=mc, nrows=mr, NODATA_value=mnd, data_format=data_format)

    # Remove the duplicated rows from the table and write it to csv:
    table.to_csv(table_path.replace('.csv', f'{output_suffix}.csv'), index=False)

## Elevation Data

Elevation data for the DEM and minDEM is taken from the OS Terrain 50 dataset. This is free to download:
https://osdatahub.os.uk/downloads/open/Terrain50

Around the coastline, the OS data shows the sea using negative values (presumably taken from a low resolution bathymetry map). It is presumed that this will not impact SHETRAN elevations going forward as the setups do not run to the coast. If much larger negative values were used (i.e. -9999) then this may have a greater impact on those coastal cells compared to the current OS values (0 to -2m or so); although these would still be unlikely to be included within the model domains.

This is used to create the DEM and minimum DEM (which is used for rivers).

OSNI 50m data for Northern Ireland was downloaded as a csv of points. These were converted into an ascii grid using QGIS:
 1. Reprojected from ING to BNG.
2. Converted from points to gridded raster with extents rounded to the appropriate 50m.
3. No data cells (where there were no points in a raster cell) were filled using Fill No Data, ensuring to only look 1 cell away for a value. This does fill some water cells that should be missing data, but this is non-consequential.
4. This filling process was repeated a few times to fill in gaps in the dats where there are lakes etc. Again, non-consequential.
5. Data written as an ascii grid for incorporation into the rasters below. You can use QGIS's _Convert Format_ with _Additional command line parameters_ '-co DECIMAL_PRECISION=1' to write this with 1 decimal place to reduce file size.
6. The NI data would not immediately merge with the GB data due to an issue with the projection. These were very similar (see below), and so I simply copied a GB projection from a prj file to the NI prj file... I don't think this makes any tangible difference.

GB Projection:
<code>
PROJCS["British_National_Grid",GEOGCS["GCS_OSGB_1936",DATUM["D_OSGB_1936",SPHEROID["Airy_1830",6377563.396,299.3249646]],PRIMEM["Greenwich",0],UNIT["Degree",0.017453292519943295]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",400000],PARAMETER["False_Northing",-100000],PARAMETER["Central_Meridian",-2],PARAMETER["Scale_Factor",0.999601272],PARAMETER["Latitude_Of_Origin",49],UNIT["Meter",1]]
</code>

Original NI Projection
<code>
PROJCS["British_National_Grid",GEOGCS["GCS_OSGB_1936",DATUM["D_OSGB_1936",SPHEROID["Airy_1830",6377563.396,299.3249646]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",400000.0],PARAMETER["False_Northing",-100000.0],PARAMETER["Central_Meridian",-2.0],PARAMETER["Scale_Factor",0.9996012717],PARAMETER["Latitude_Of_Origin",49.0],UNIT["Meter",1.0]]
</code>



In [ ]:
# Unzip the OS download:
OS50_zip_path = os.path.join(root, "01 - Downloaded Data", "Elevation Data", "terr50_gagg_gb.zip")  # /data/
OS50_unzip_path = os.path.join(os.path.dirname(OS50_zip_path), 'Unzipped_data')
with zipfile.ZipFile(OS50_zip_path, 'r') as zip_ref:
            zip_ref.extractall(os.path.join(os.path.dirname(OS50_zip_path), 'Unzipped_data'))

# The data is within sub-folders, list the subfolders (tiles):
data_zip_path = os.path.join(os.path.dirname(OS50_zip_path), 'Unzipped_data', 'data')
tile_zip_folders = os.listdir(data_zip_path)

# Setup a new folder to hold the unzipped data:
OS50_unzipped_folder = os.path.join(OS50_unzip_path, 'Collated_data')
if not os.path.exists(OS50_unzipped_folder):
    os.mkdir(OS50_unzipped_folder)

# Unzip the data:
for tile_zip_folder in tile_zip_folders:
    subtile_folders = os.listdir(os.path.join(data_zip_path, tile_zip_folder))
    for zip_folder in subtile_folders:
        print(zip_folder)
        with zipfile.ZipFile(os.path.join(data_zip_path, tile_zip_folder, zip_folder), 'r') as zip_ref:
            zip_ref.extractall(OS50_unzipped_folder)

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'I:/SHETRAN_GB_2021/02_Input_Data/01 - National Data Inputs for SHETRAN UK/01 - Downloaded Data\\Elevation Data\\terr50_gagg_gb/data/'

Join the elevation rasters into a single file.

In [ ]:
# List all .asc files in the folder
asc_files = [os.path.join(OS50_unzipped_folder, f) for f in os.listdir(OS50_unzipped_folder) if f.endswith('.asc')]

# Open the GB files using rasterio:
count = 1
raster_list = []
for asc_file in asc_files:
    print(count, "/", len(asc_files))
    raster = rasterio.open(asc_file,)
    raster_list.append(raster)
    count += 1

# ---

# Open the filled NI file using rasterio:
print('NI', "/", len(asc_files))
raster = rasterio.open(os.path.join(root, 'OSNI_OpenData_50m/OSNI_OpenData_50m_BNG_Filled.asc'),)
raster_list.append(raster)

# ---

# Combine (merge) the rasters:
merged_raster, merged_transform = merge(raster_list, nodata=-9999)

# Close the opened raster files - you may be able to incorporate this into the loop above.
for raster in raster_list:
    raster.close()

# Extract the first raster band and change 0s to -9999:
merged_raster = merged_raster[0]
# merged_raster[merged_raster == 0] = -9999  # This was changed to merge(..., nodata=-9999) as it created issues in the fens

National_OS50_path = os.path.join(root, 'Processed Data', 'National_OS50.asc')

# Write the file as an ascii:
she.write_ascii(
    array=merged_raster,
    ascii_ouput_path=National_OS50_path,
    xllcorner=merged_transform[2],
    yllcorner=merged_transform[5]-(merged_raster.shape[0]*merged_transform[0]),
    cellsize=merged_transform[0],
)

Regrid the elevation rasters to the desired size.

Note that this does assume that the lower left corner of the national OS50 file is at 0,0, easting northing. Check this if you are redoing this work. you can load the header of the file using the following code:
<code>
headers = []
with open(OS50_zip_path + 'National_OS50.asc', 'r') as fh:
for i in range(6):
asc_line = fh.readline()
headers.append(asc_line.rstrip())
headers
</code>

The first stage of this is to ensure that the 50m data is of the same extent as the 1km data. Rows and columns are added to ensure this. This means that the data has an extent that is in 1km, so can be resampled to divisions of this (1km, 500m, 200m, 100m). This may not work if you try other resolutions as, because the calculations will run from the top left, not the bottom left, the resampled dataset may not have llx/lly coordinates of 0,0. Think about this if you want to use other resolutions!

In [ ]:
national_OS50, _, _, _, _, _, _, _, OS50_header = she.read_ascii_raster(National_OS50_path, data_type=float, replace_NA=True)

FileNotFoundError: [Errno 2] No such file or directory: 'I:/SHETRAN_GB_2021/02_Input_Data/01 - National Data Inputs for SHETRAN UK/Processed Data\\National_OS50.asc'

In [ ]:
# # If you have not loaded in the dataset (perhaps because you are only testing the code), you can check the dimensions of the 50m dataset using this code:
#
# OS50_header = {}
# with open(OS50_zip_path + 'National_OS50.asc', 'r') as fh:
#     for i in range(6):
#         asc_line = fh.readline()
#         key, val = asc_line.rstrip().split()
#         OS50_header[key] = val
# OS50_header

Resize the national dataset to match existing SHETRAN inputs:

In [ ]:
# Resize the inputs to the desired SHETRAN grid (top right corner should be x: 661000, y: 1241000):
row_difference = int(((1241*1000) - float(OS50_header['nrows']) * float(OS50_header['cellsize'])) / float(OS50_header['cellsize']))
col_difference = int(((661*1000) - float(OS50_header['ncols']) * float(OS50_header['cellsize'])) / float(OS50_header['cellsize']))

if row_difference > 0:
    # Create the rows of -9999
    new_rows = np.full((row_difference, national_OS50.shape[1]), -9999)
    # Add the new rows to the top
    national_OS50 = np.vstack((new_rows, national_OS50))

# repeat for columns:
if row_difference > 0:
    new_cols = np.full((national_OS50.shape[0], col_difference), -9999)
    national_OS50 = np.hstack((national_OS50, new_cols))  # Remember that these need adding at the end/right.

_I have removed the code chuck below as I think it is superfluous. There were some issues resulting from changing 0 values to NA when in fact these are valid elevations. This has been corrected and the code designed to fix/fill the holes left below in case of potential future uses._

*it may be of use in the Northern Ireland catchments, where there is a greater presence of NA values over lakes.*

_This will fill the holes (na/-9999 values) in the dataset - this code will only fill calls that have a valid value in an adjacent cell._

<code>
\# Replace hole_value with NaN for processing
raster[raster == -9999] = np.nan
\# Apply the function iteratively
filled_national_OS50 = generic_filter(national_OS50, fill_holes, size=3, mode='constant', cval=np.nan)
filled_national_OS50[filled_national_OS50 == np.nan] = -9999
\# Write the file as an ascii:
write_ascii(
    array=filled_national_OS50,
    ascii_ouput_path=f'{OS50_zip_path}National_OS50_DEM_preprocessed.asc',
    xllcorner=OS50_header['xllcorner'],
    yllcorner=OS50_header['yllcorner'],
    cellsize=float(OS50_header['cellsize'])
)
</code>

**The following code will give warnings when trying to take the mean of cells that are all np.nan - don't worry, this is doing what it should. (Probably check everything in QGIS or similar at the end though).**

In [ ]:
# Define the block size for aggregation
resolution_input = float(OS50_header['cellsize'])
block_size = int(resolution_output/resolution_input)  # For 50m -> 100m, use a block size of 2

# Resample using the mean and minimum:
DEM = she.cell_reduce_with_pad(national_OS50, block_size, np.mean)
minDEM = she.cell_reduce_with_pad(national_OS50, block_size, np.min)

# -9999 was converted to np.nan in the loading phase, convert it back
DEM[np.isnan(DEM)] = -9999
minDEM[np.isnan(minDEM)] = -9999

In [ ]:
# Write the DEM file as an ascii:
she.write_ascii(
    array=DEM,
    ascii_ouput_path=f'{root}/Processed Data/National_OS50_DEM_{resolution_output}m.asc',
    xllcorner=OS50_header['xllcorner'],
    yllcorner=OS50_header['yllcorner'],
    cellsize=resolution_output
)

# Write the minDEM file as an ascii:
she.write_ascii(
    array=minDEM,
    ascii_ouput_path=f'{root}/Processed Data/National_OS50_minDEM_{resolution_output}m.asc',
    xllcorner=OS50_header['xllcorner'],
    yllcorner=OS50_header['yllcorner'],
    cellsize=resolution_output
)
